### Using XLNet to generate strange stories.
this notebook illustrates text generation in the most basic way.  Notice the generator invoked a draw from a multinomial distribution to take the next word.  this means that you will never get the same story twice.
You will also notice that it gets stuck in a loop of symbols and result is garbage. 

In [3]:
import string
import re
import os
import sys
import csv
import logging
import numpy as np
import torch
import pandas as pd

In [2]:
#!pip install transformers

In [1]:
from transformers import XLNetLMHeadModel, XLNetTokenizer

In [4]:
tokenizer3 = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model3 = XLNetLMHeadModel.from_pretrained('xlnet-base-cased')

In [12]:
PADDING_TEXT = """ In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""
print(len(PADDING_TEXT))

718


In [5]:
import torch.nn.functional as F

In [6]:
def make_up_story(st, length):
    st1 = PADDING_TEXT + st
    context_tokens = tokenizer3.encode(st1)
    context = torch.tensor(context_tokens, dtype=torch.long)
    context = context.unsqueeze(0).repeat(1,1)
    generated = context

    with torch.no_grad():
        for _ in range(length):
            #add a blank token at the end
            input_ids = torch.cat((generated, torch.zeros((1, 1), dtype=torch.long)), dim=1)
            #create the permutation and target_mask
            perm_mask = torch.zeros((1,input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
            perm_mask[:, :, -1] = 1.0
            target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)
            target_mapping[0, 0, -1] = 1.0 

            outputs = model3(input_ids=input_ids, perm_mask=perm_mask, 
                             target_mapping=target_mapping)

            next_token_logits = outputs[0][0, -1, :]
            next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
    out = generated[0, len(context_tokens):].tolist()
    text = tokenizer3.decode(out, clean_up_tokenization_spaces=True)
    print(st+' '+text)

In [8]:
make_up_story("General relativity has been seen to describe black holes and", 100)

General relativity has been seen to describe black holes and <cls> is a power of quantum physics to describe black holes and other red-hot "black holes". Within 3 or 2 dimensions, black holes have been shown to be "black hole" of the infinite universe, which is one of the many causal forces which drives all other Universes. They are also today considered to be " black hole of the " Universe, which is the largest wave on the entire planet felt for the entire Universe. In incorporate that detail, he metaphorically described


In [13]:
make_up_story('Martha wanted Jim to fly the space ship, but he said he would need more ray guns and fuel. The', 100)

Martha wanted Jim to fly the space ship, but he said he would need more ray guns and fuel. The to be stuck on ice was Ty Var. He said that the projected 1,000 miles away star would travel back through space and get thermal rays from the assembly of ray guns. Ty Var wanted him to fly the space ship, but he said he had more fuel to use. Long deployment will be needed to create a stable orbit for the space breed. Matthew 55 15:247-28.<eop> Vietnamese Tom Tay (21:9-12) Early Fruit Order Structures Education Use-


In [18]:
make_up_story('He studied Planetary Science and Biology in school', 100)

He studied Planetary Science and Biology in school Sulening where, before a fifth grade age, he helped to train the group for flying. Scrush-box matching with Danny Attha of Fox and the White Rabbit, Rasputin learned flying principles, formidable re-learning, jumping and pulling. After his Top Bubble for flight test, Rasputin was rewarded with a ten million ruble scholarship. For his work, Rasputin also received ten million rubles, including an exclusive service for his families, including one including two girls


In [7]:
#experiment with a different padding text
PADDING_TEXT = """
In ancient manuscripts, another means to divide sentences into paragraphs was a line break (newline) 
followed by an initial at the beginning of the next paragraph. An initial is an oversized capital letter, 
sometimes outdented beyond the margin of the text. This style can be seen, for example, in the original 
Old English manuscript of Beowulf. Outdenting is still used in English typography, though not commonly.
Modern English typography usually indicates a new paragraph by indenting the first line. 
This style can be seen in the (handwritten) United States Constitution from 1787. 
For additional ornamentation, a hedera leaf or other symbol can be added to the inter-paragraph whitespace, 
or put in the indentation space. <eod> </s> <eos>"""


In [20]:
make_up_story("DNA describes the unique nature of each indivual and", 100)

DNA describes the unique nature of each indivual and .<eop><eod> Leaders discuss the federal involvement in establishing political and economic institutions in the state councils and the national assembly, and the importance of professional education for that state. Members will also analyze how the role of corporate responses in developing new businesses is being appropriate in our area.<eop><eod> Suppliers will refuse to accept non-SOLO certified products at any time for further study. All products are tracked by Data File Dr. PS (as implemented by SAP, Data Chemical// Ethernet) as written


In [21]:
make_up_story("David and Sarah", 100)

David and Sarah top certificate in arts and sciences for the award for outstanding work. With tour operators, students and administrators, Geoff and Sarah are served with awards, achievements, rankings and list makers.<eop><eod> Earn cash positions for the acquisition of your n-phone card with Easy Sales... Apply today! 5. Get a discount off your n-phone card rates! 7. Earn new cash positions from our rate guides and add them to your account. 8. ( transmit as an anonymous mailing


In [24]:
make_up_story("David and Sarah came over the hill and saw the dinasaur.", 100)

David and Sarah came over the hill and saw the dinasaur. get it for sale here in Jamaica? In Jamaica, how do you find out. You have to pay out your local government for its license to sell a the dinasaur. People from school and a community can only purchase it on bare bones, such as lined wooden cloth. No concession from the state process. So finally, we are currently looking at buy a local dinasaur just for governmental service. And if we don't get adoption, so what do we do with that money


In [25]:
make_up_story("General relativity has been seen to describe black holes and", 100)

General relativity has been seen to describe black holes and not get near these dark regions for many reasons. The dark regions are a very remote region like the void or a difficult distance from the physical world. However, members of the quantum psychologist and quantum physicist college also biggest contributions to quantum physicist physics may be present in their original studies and work.<eop> The collection is not specific to quantum psychology, but as a matter may help with considering related questions about the methods of relativity. This collections are about quantum psychology, whose big contributions to


In [26]:
make_up_story('He studied Planetary Science and Biology in school', 100)

He studied Planetary Science and Biology in school competition and marathons with high school runner T. Dick Moe. He ran three races between his college and high school years. In ninth grade, he had short bursts of running. He sometimes didn't get his his properly, and he absolutely hated the feeling of not being in the right place in a field he was commanding.<eop> Hiring unmanned aerial vehicles. He studied unmanned aerial vehicles, in other words, robotic platforms on the ground, running around the space to their maximum accuracy.


In [10]:
make_up_story("General relativity describes black holes and gravity waves. Hence ", 100)

General relativity describes black holes and gravity waves. Hence  an odd one, with a hard and rugged physical society, meaning of also being the early as a Latin well suited for appealing to the northern population.<eop> Cheap Faso Cake Brand Cheap Faso Cake Brand Eli.... Order.... You Need to feel treated fairly... Buy.... polish is popular to southern people... Buy, or buy a filter, the same way as your price is high on any filter, because it's easier to "earn
